In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.constant([1, 2, 3])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [4]:
tf.constant(((1, 2, 3), (1, 2, 3)))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [1, 2, 3]], dtype=int32)>

In [5]:
arr = np.array([1, 2, 3])
arr

array([1, 2, 3])

In [7]:
tensor = tf.constant(arr)
tensor

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 2, 3])>

In [8]:
tensor.shape

TensorShape([3])

In [9]:
tensor.dtype

tf.int64

In [10]:
tensor = tf.constant(arr, dtype=tf.float32)
tensor

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [11]:
arr = np.array([1, 2, 3], dtype=np.float32)
arr.astype(np.uint8)

array([1, 2, 3], dtype=uint8)

In [12]:
tensor

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [13]:
tf.cast(tensor, dtype=tf.uint8)

<tf.Tensor: shape=(3,), dtype=uint8, numpy=array([1, 2, 3], dtype=uint8)>

In [14]:
tensor.numpy()

array([1., 2., 3.], dtype=float32)

In [15]:
np.array(tensor)

array([1., 2., 3.], dtype=float32)

### 난수 생성

In [16]:
np.random.randn(9)

array([ 0.8676764 , -0.09352617,  0.43716527, -0.7223949 ,  0.47231536,
        0.51642203, -1.72970863,  0.50569757, -0.72152741])

In [17]:
np.random.normal(9)

8.843757767334433

In [19]:
tf.random.normal([3, 3])

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.45970613, -0.01015105, -1.822091  ],
       [ 1.7721825 , -1.3910561 , -1.8653673 ],
       [ 0.7362502 ,  0.27782518,  0.5809572 ]], dtype=float32)>

In [20]:
tf.random.uniform([4, 4])

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0.53726256, 0.7029269 , 0.24778819, 0.59721506],
       [0.8671092 , 0.8191887 , 0.35200393, 0.87058294],
       [0.074417  , 0.93916297, 0.3604865 , 0.83665514],
       [0.24559045, 0.7560705 , 0.76195073, 0.7941512 ]], dtype=float32)>

## 하이퍼 파라미터 설정

In [21]:
EPOCHS = 5

### 네트워크 구조 정의

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

### 학습 루프 정의

In [26]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)        

### 테스트 루프 정의

In [27]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)        

### 데이터셋 가져오고, 정리

In [28]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 네트워크(모델) 생성

In [30]:
#create model
model = MyModel()

### loss function, optimizer 정의

In [31]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### performance metrics 정의

In [32]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### 알고리즘 학습, 테스트 루프

In [34]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels, in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result() * 100,
                         test_loss.result(),
                         test_accuracy.result() * 100))

Epoch 1, Loss: 0.29837024211883545, Accuracy: 91.9816665649414, Test Loss: 0.16698455810546875, Test Accuracy: 95.1300048828125
Epoch 2, Loss: 0.2464163899421692, Accuracy: 93.29444122314453, Test Loss: 0.15132565796375275, Test Accuracy: 95.5
Epoch 3, Loss: 0.21232686936855316, Accuracy: 94.1824951171875, Test Loss: 0.13928313553333282, Test Accuracy: 95.82666778564453
Epoch 4, Loss: 0.1875588446855545, Accuracy: 94.83433532714844, Test Loss: 0.12854811549186707, Test Accuracy: 96.10749816894531
Epoch 5, Loss: 0.16832631826400757, Accuracy: 95.35944366455078, Test Loss: 0.12109764665365219, Test Accuracy: 96.302001953125
